## 📦 1. IMPORTS ET CHARGEMENT DU MODÈLE

In [40]:
import pandas as pd
import numpy as np
import pickle
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("✅ Imports réussis")

✅ Imports réussis


In [41]:
# Charger le modèle et les encodeurs
print("📥 Chargement du modèle et des encodeurs...")

with open('models/extra_trees_tuned.pkl', 'rb') as f:
    model = pickle.load(f)
print("✅ Modèle Extra Trees chargé")

with open('models/encoders.pkl', 'rb') as f:
    encoders = pickle.load(f)
print("✅ Encodeurs chargés")

print(f"\n📊 Modèle: {type(model).__name__}")
print(f"📊 Encodeurs disponibles: {list(encoders.keys())}")

📥 Chargement du modèle et des encodeurs...
✅ Modèle Extra Trees chargé
✅ Encodeurs chargés

📊 Modèle: ExtraTreesRegressor
📊 Encodeurs disponibles: ['energie_columns', 'brand_category_columns', 'age_category_columns', 'marque_encoder']


## 🛠️ 2. PIPELINE DE TRANSFORMATION DES DONNÉES

In [42]:
# Liste des marques acceptées
MARQUES_ACCEPTEES = [
    'MERCEDES', 'VW', 'PEUGEOT', 'KIA', 'CITROEN', 'BMW', 'Fiat', 'OTHER_BRAND',
    'Audi', 'CHINESE', 'HYUNDAI', 'Toyota', 'SUZUKI', 'RENAULT', 'Dacia',
    'JAPANESE', 'Ford', 'MG', 'GWM', 'SEAT', 'AMERICAN', 'NISSAN', 'CHERY',
    'SKODA', 'Porsche', 'LUXURY_BRAND', 'Opel', 'Mini', 'Land Rover', 'UTILITY'
]

# Marques de luxe
LUXURY_BRANDS = ['BMW', 'MERCEDES', 'Audi', 'Porsche', 'Land Rover', 'LUXURY_BRAND', 'Mini']

# Catégories de marques
BRAND_CATEGORIES = {
    'Economic_European': ['PEUGEOT', 'CITROEN', 'RENAULT', 'Fiat', 'SEAT', 'Dacia', 'Opel', 'SKODA', 'Ford'],
    'Premium_European': ['BMW', 'MERCEDES', 'Audi', 'VW', 'Porsche', 'Land Rover', 'Mini', 'LUXURY_BRAND'],
    'Asian': ['Toyota', 'HYUNDAI', 'KIA', 'SUZUKI', 'NISSAN', 'JAPANESE'],
    'Chinese': ['CHINESE', 'MG', 'GWM', 'CHERY'],
    'Other': ['OTHER_BRAND', 'AMERICAN', 'UTILITY']
}

print(f"✅ {len(MARQUES_ACCEPTEES)} marques configurées")
print(f"✅ {len(LUXURY_BRANDS)} marques de luxe")
print(f"✅ {len(BRAND_CATEGORIES)} catégories de marques")

✅ 30 marques configurées
✅ 7 marques de luxe
✅ 5 catégories de marques


In [43]:
def age_category(age):
    """Catégorise un véhicule selon son âge"""
    if age == 0:
        return 'Neuf'
    elif age <= 3:
        return 'Récent'
    elif age <= 7:
        return 'Occasion_Standard'
    else:
        return 'Ancien'

def categorize_brand(marque):
    """Catégorise une marque selon son origine"""
    for category, brands in BRAND_CATEGORIES.items():
        if marque in brands:
            return category
    return 'Other'

print("✅ Fonctions de catégorisation définies")

✅ Fonctions de catégorisation définies


In [44]:
def prepare_input_data(marque, modele, annee, kilometrage, energie, boite_vitesses, puissance_fiscale):
    """
    Prépare les données d'entrée utilisateur pour le modèle.
    
    Parameters:
    -----------
    marque : str
        La marque du véhicule (doit être dans MARQUES_ACCEPTEES)
    modele : str
        Le modèle du véhicule (informatif, non utilisé par le modèle)
    annee : int
        L'année de fabrication (ex: 2020)
    kilometrage : float
        Le kilométrage du véhicule
    energie : str
        Type d'énergie ('Diesel', 'Essence', 'Hybride', 'Electrique', 'GPL')
    boite_vitesses : str
        Type de boîte ('Manuelle' ou 'Automatique')
    puissance_fiscale : int
        Puissance fiscale du véhicule
    
    Returns:
    --------
    pd.DataFrame
        DataFrame avec toutes les features nécessaires pour le modèle (23 colonnes)
    """
    
    # Validation de la marque
    if marque not in MARQUES_ACCEPTEES:
        raise ValueError(f"Marque '{marque}' non reconnue. Marques acceptées: {MARQUES_ACCEPTEES}")
    
    # Validation de l'énergie
    energies_acceptees = ['Diesel', 'Essence', 'Hybride', 'Electrique', 'GPL']
    if energie not in energies_acceptees:
        raise ValueError(f"Énergie '{energie}' non reconnue. Valeurs acceptées: {energies_acceptees}")
    
    # Validation de la boîte de vitesses
    boites_acceptees = ['Manuelle', 'Automatique']
    if boite_vitesses not in boites_acceptees:
        raise ValueError(f"Boîte de vitesses '{boite_vitesses}' non reconnue. Valeurs acceptées: {boites_acceptees}")
    
    # Calcul de l'âge
    annee_actuelle = datetime.now().year
    age = annee_actuelle - annee
    
    if age < 0:
        raise ValueError(f"L'année {annee} est dans le futur!")
    
    print("="*70)
    print("🔧 TRANSFORMATION DES DONNÉES")
    print("="*70)
    print(f"📅 Année actuelle: {annee_actuelle}")
    print(f"📅 Année véhicule: {annee}")
    print(f"📅 Âge calculé: {age} ans")
    
    # ========================================================================
    # FEATURE ENGINEERING (identique au training)
    # ========================================================================
    
    # 1. Km par âge
    km_par_age = kilometrage / (age + 1)
    
    # 2. Log kilométrage
    log_km = np.log1p(kilometrage)
    
    # 3. Is Luxury
    is_luxury = 1 if marque in LUXURY_BRANDS else 0
    
    # 4. Age Category
    age_cat = age_category(age)
    
    # 5. Puissance Age Ratio
    puissance_age_ratio = puissance_fiscale / (age + 1)
    
    # 6. Brand Category
    brand_cat = categorize_brand(marque)
    
    print(f"\n✅ Features engineered créées:")
    print(f"   • Km_par_Age: {km_par_age:,.0f} km/an")
    print(f"   • Log_Km: {log_km:.2f}")
    print(f"   • Is_Luxury: {'Oui' if is_luxury else 'Non'}")
    print(f"   • Age_Category: {age_cat}")
    print(f"   • Puissance_Age_Ratio: {puissance_age_ratio:.2f}")
    print(f"   • Brand_Category: {brand_cat}")
    
    # ========================================================================
    # ENCODAGE (ordre EXACT du training notebook)
    # ========================================================================
    
    # Label Encoding pour Marque
    le_marque = encoders['marque_encoder']
    try:
        marque_encoded = le_marque.transform([marque])[0]
    except ValueError:
        # Si la marque n'est pas dans l'encoder, utiliser une valeur par défaut
        print(f"⚠️  Marque '{marque}' non trouvée dans l'encoder, utilisation de 'OTHER_BRAND'")
        marque_encoded = le_marque.transform(['OTHER_BRAND'])[0]
    
    # ORDRE EXACT DU TRAINING: Age, Kilometrage, Puissance_Fiscale,
    #                         Km_par_Age, Log_Km, Is_Luxury, Puissance_Age_Ratio,
    #                         Boite_Auto, Marque_encoded,
    #                         + energie_columns + brand_category_columns + age_category_columns
    
    data = {
        # 1. Features numériques de base (3)
        'Age': age,
        'Kilometrage': kilometrage,
        'Puissance_Fiscale': puissance_fiscale,
        
        # 2. Features engineered (4) - ORDRE IMPORTANT: Km_par_Age, Log_Km, Is_Luxury, Puissance_Age_Ratio
        'Km_par_Age': km_par_age,
        'Log_Km': log_km,
        'Is_Luxury': is_luxury,
        'Puissance_Age_Ratio': puissance_age_ratio,
        
        # 3. Features encodées (2)
        'Boite_Auto': 1 if boite_vitesses == 'Automatique' else 0,
        'Marque_encoded': marque_encoded
    }
    
    # 4. One-Hot Encoding pour Énergie (5 colonnes)
    for col in encoders['energie_columns']:
        energie_type = col.replace('Energie_', '')
        data[col] = 1 if energie_type == energie else 0
    
    # 5. One-Hot Encoding pour Brand_Category (5 colonnes)
    for col in encoders['brand_category_columns']:
        brand_cat_type = col.replace('Brand_Cat_', '')
        data[col] = 1 if brand_cat_type == brand_cat else 0
    
    # 6. One-Hot Encoding pour Age_Category (4 colonnes)
    for col in encoders['age_category_columns']:
        age_cat_type = col.replace('Age_Cat_', '')
        data[col] = 1 if age_cat_type == age_cat else 0
    
    # Créer le DataFrame
    df_input = pd.DataFrame([data])
    
    print(f"\n✅ DataFrame créé: {df_input.shape[1]} colonnes")
    print(f"   📋 Ordre des colonnes vérifié avec training notebook")
    print(f"   📋 Features: Age, Kilometrage, Puissance_Fiscale, Km_par_Age, Log_Km, Is_Luxury,")
    print(f"                Puissance_Age_Ratio, Boite_Auto, Marque_encoded, +14 One-Hot")
    
    return df_input

print("✅ Fonction prepare_input_data définie")

✅ Fonction prepare_input_data définie


## 🎯 3. FONCTION DE PRÉDICTION

In [45]:
def predict_price(marque, modele, annee, kilometrage, energie, boite_vitesses, puissance_fiscale, verbose=True):
    """
    Prédit le prix d'un véhicule en utilisant le modèle Extra Trees.
    
    Parameters:
    -----------
    marque : str
        La marque du véhicule
    modele : str
        Le modèle du véhicule (informatif)
    annee : int
        L'année de fabrication
    kilometrage : float
        Le kilométrage du véhicule
    energie : str
        Type d'énergie
    boite_vitesses : str
        Type de boîte
    puissance_fiscale : int
        Puissance fiscale
    verbose : bool, optional
        Afficher les détails (default: True)
    
    Returns:
    --------
    dict
        Dictionnaire avec le prix prédit et les informations du véhicule
    """
    
    if verbose:
        print("="*70)
        print("🚗 PRÉDICTION DU PRIX")
        print("="*70)
        print(f"\n📋 INFORMATIONS DU VÉHICULE:")
        print(f"   • Marque:           {marque}")
        print(f"   • Modèle:           {modele}")
        print(f"   • Année:            {annee}")
        print(f"   • Kilométrage:      {kilometrage:,.0f} km")
        print(f"   • Énergie:          {energie}")
        print(f"   • Boîte:            {boite_vitesses}")
        print(f"   • Puissance:        {puissance_fiscale} CV")
    
    try:
        # Préparer les données
        df_input = prepare_input_data(
            marque, modele, annee, kilometrage, 
            energie, boite_vitesses, puissance_fiscale
        )
        
        # Faire la prédiction
        if verbose:
            print("\n🔮 Prédiction en cours...")
        
        # Le modèle prédit directement le prix (pas de log-transform)
        prix_predit = model.predict(df_input)[0]
        
        if verbose:
            print(f"   • Prix prédit par le modèle: {prix_predit:,.0f} DT")
        
        # Calculer une fourchette de prix (±10% pour incertitude)
        prix_min = prix_predit * 0.90
        prix_max = prix_predit * 1.10
        
        if verbose:
            print("\n" + "="*70)
            print("💰 RÉSULTAT DE LA PRÉDICTION")
            print("="*70)
            print(f"\n🎯 Prix estimé:     {prix_predit:,.0f} DT")
            print(f"\n📊 Fourchette:      {prix_min:,.0f} DT - {prix_max:,.0f} DT")
            print(f"   (±10% d'incertitude)")
            print("\n" + "="*70)
        
        return {
            'success': True,
            'prix_predit': prix_predit,
            'prix_min': prix_min,
            'prix_max': prix_max,
            'marque': marque,
            'modele': modele,
            'annee': annee,
            'age': datetime.now().year - annee,
            'kilometrage': kilometrage,
            'energie': energie,
            'boite_vitesses': boite_vitesses,
            'puissance_fiscale': puissance_fiscale
        }
        
    except Exception as e:
        if verbose:
            print(f"\n❌ ERREUR: {str(e)}")
        
        return {
            'success': False,
            'error': str(e)
        }

print("✅ Fonction predict_price définie")

✅ Fonction predict_price définie


## 🧪 4. TESTS DE FONCTIONNALITÉ

### Test 1: Véhicule Premium Récent

In [46]:
# Test 1: BMW Série 3 récente
result1 = predict_price(
    marque='BMW',
    modele='Série 1',
    annee=2020,
    kilometrage=130000,
    energie='Essence',
    boite_vitesses='Automatique',
    puissance_fiscale=10
)

if result1['success']:
    print(f"\n✅ Test 1 réussi: Prix prédit = {result1['prix_predit']:,.0f} DT")
    print(f"   Fourchette: {result1['prix_min']:,.0f} - {result1['prix_max']:,.0f} DT")
else:
    print(f"\n❌ Test 1 échoué: {result1['error']}")

🚗 PRÉDICTION DU PRIX

📋 INFORMATIONS DU VÉHICULE:
   • Marque:           BMW
   • Modèle:           Série 1
   • Année:            2020
   • Kilométrage:      130,000 km
   • Énergie:          Essence
   • Boîte:            Automatique
   • Puissance:        10 CV
🔧 TRANSFORMATION DES DONNÉES
📅 Année actuelle: 2025
📅 Année véhicule: 2020
📅 Âge calculé: 5 ans

✅ Features engineered créées:
   • Km_par_Age: 21,667 km/an
   • Log_Km: 11.78
   • Is_Luxury: Oui
   • Age_Category: Occasion_Standard
   • Puissance_Age_Ratio: 1.67
   • Brand_Category: Premium_European

✅ DataFrame créé: 23 colonnes
   📋 Ordre des colonnes vérifié avec training notebook
   📋 Features: Age, Kilometrage, Puissance_Fiscale, Km_par_Age, Log_Km, Is_Luxury,
                Puissance_Age_Ratio, Boite_Auto, Marque_encoded, +14 One-Hot

🔮 Prédiction en cours...
   • Prix prédit par le modèle: 133,828 DT

💰 RÉSULTAT DE LA PRÉDICTION

🎯 Prix estimé:     133,828 DT

📊 Fourchette:      120,445 DT - 147,210 DT
   (±10% d'inc

### Test 2: Véhicule Économique d'Occasion

In [33]:
# Test 2: Peugeot 208 d'occasion
result2 = predict_price(
    marque='PEUGEOT',
    modele='208',
    annee=2021,
    kilometrage=119000,
    energie='Diesel',
    boite_vitesses='Manuelle',
    puissance_fiscale=5
)

if result2['success']:
    print(f"\n✅ Test 2 réussi: Prix prédit = {result2['prix_predit']:,.0f} DT")
    print(f"   Fourchette: {result2['prix_min']:,.0f} - {result2['prix_max']:,.0f} DT")
else:
    print(f"\n❌ Test 2 échoué: {result2['error']}")

🚗 PRÉDICTION DU PRIX

📋 INFORMATIONS DU VÉHICULE:
   • Marque:           PEUGEOT
   • Modèle:           208
   • Année:            2021
   • Kilométrage:      119,000 km
   • Énergie:          Diesel
   • Boîte:            Manuelle
   • Puissance:        5 CV
🔧 TRANSFORMATION DES DONNÉES
📅 Année actuelle: 2025
📅 Année véhicule: 2021
📅 Âge calculé: 4 ans

✅ Features engineered créées:
   • Km_par_Age: 23,800 km/an
   • Log_Km: 11.69
   • Is_Luxury: Non
   • Age_Category: Occasion_Standard
   • Puissance_Age_Ratio: 1.00
   • Brand_Category: Economic_European

✅ DataFrame créé: 23 colonnes
   📋 Ordre des colonnes vérifié avec training notebook
   📋 Features: Age, Kilometrage, Puissance_Fiscale, Km_par_Age, Log_Km, Is_Luxury,
                Puissance_Age_Ratio, Boite_Auto, Marque_encoded, +14 One-Hot

🔮 Prédiction en cours...
   • Prix prédit par le modèle: 49,204 DT

💰 RÉSULTAT DE LA PRÉDICTION

🎯 Prix estimé:     49,204 DT

📊 Fourchette:      44,284 DT - 54,125 DT
   (±10% d'incertitude

### Test 3: Véhicule Neuf

In [34]:
# Test 3: Toyota Corolla neuve
result3 = predict_price(
    marque='Toyota',
    modele='Corolla',
    annee=2025,
    kilometrage=50,
    energie='Hybride',
    boite_vitesses='Automatique',
    puissance_fiscale=7
)

if result3['success']:
    print(f"\n✅ Test 3 réussi: Prix prédit = {result3['prix_predit']:,.0f} DT")
    print(f"   Fourchette: {result3['prix_min']:,.0f} - {result3['prix_max']:,.0f} DT")
else:
    print(f"\n❌ Test 3 échoué: {result3['error']}")

🚗 PRÉDICTION DU PRIX

📋 INFORMATIONS DU VÉHICULE:
   • Marque:           Toyota
   • Modèle:           Corolla
   • Année:            2025
   • Kilométrage:      50 km
   • Énergie:          Hybride
   • Boîte:            Automatique
   • Puissance:        7 CV
🔧 TRANSFORMATION DES DONNÉES
📅 Année actuelle: 2025
📅 Année véhicule: 2025
📅 Âge calculé: 0 ans

✅ Features engineered créées:
   • Km_par_Age: 50 km/an
   • Log_Km: 3.93
   • Is_Luxury: Non
   • Age_Category: Neuf
   • Puissance_Age_Ratio: 7.00
   • Brand_Category: Asian

✅ DataFrame créé: 23 colonnes
   📋 Ordre des colonnes vérifié avec training notebook
   📋 Features: Age, Kilometrage, Puissance_Fiscale, Km_par_Age, Log_Km, Is_Luxury,
                Puissance_Age_Ratio, Boite_Auto, Marque_encoded, +14 One-Hot

🔮 Prédiction en cours...
   • Prix prédit par le modèle: 145,429 DT

💰 RÉSULTAT DE LA PRÉDICTION

🎯 Prix estimé:     145,429 DT

📊 Fourchette:      130,886 DT - 159,972 DT
   (±10% d'incertitude)


✅ Test 3 réussi: Pri

### Test 4: Véhicule Chinois Peu Cher

In [36]:
# Test 4: MG ZS
result4 = predict_price(
    marque='MG',
    modele='ZS',
    annee=2022,
    kilometrage=30000,
    energie='Essence',
    boite_vitesses='Automatique',
    puissance_fiscale=6
)

if result4['success']:
    print(f"\n✅ Test 4 réussi: Prix prédit = {result4['prix_predit']:,.0f} DT")
    print(f"   Fourchette: {result4['prix_min']:,.0f} - {result4['prix_max']:,.0f} DT")
else:
    print(f"\n❌ Test 4 échoué: {result4['error']}")

🚗 PRÉDICTION DU PRIX

📋 INFORMATIONS DU VÉHICULE:
   • Marque:           MG
   • Modèle:           ZS
   • Année:            2022
   • Kilométrage:      30,000 km
   • Énergie:          Essence
   • Boîte:            Automatique
   • Puissance:        6 CV
🔧 TRANSFORMATION DES DONNÉES
📅 Année actuelle: 2025
📅 Année véhicule: 2022
📅 Âge calculé: 3 ans

✅ Features engineered créées:
   • Km_par_Age: 7,500 km/an
   • Log_Km: 10.31
   • Is_Luxury: Non
   • Age_Category: Récent
   • Puissance_Age_Ratio: 1.50
   • Brand_Category: Chinese

✅ DataFrame créé: 23 colonnes
   📋 Ordre des colonnes vérifié avec training notebook
   📋 Features: Age, Kilometrage, Puissance_Fiscale, Km_par_Age, Log_Km, Is_Luxury,
                Puissance_Age_Ratio, Boite_Auto, Marque_encoded, +14 One-Hot

🔮 Prédiction en cours...
   • Prix prédit par le modèle: 68,261 DT

💰 RÉSULTAT DE LA PRÉDICTION

🎯 Prix estimé:     68,261 DT

📊 Fourchette:      61,435 DT - 75,088 DT
   (±10% d'incertitude)


✅ Test 4 réussi: Prix

### Test 5: Véhicule de Luxe Ancien

In [37]:
# Test 5: Mercedes Classe E ancienne
result5 = predict_price(
    marque='MERCEDES',
    modele='Classe E',
    annee=2015,
    kilometrage=180000,
    energie='Diesel',
    boite_vitesses='Automatique',
    puissance_fiscale=11
)

if result5['success']:
    print(f"\n✅ Test 5 réussi: Prix prédit = {result5['prix_predit']:,.0f} DT")
    print(f"   Fourchette: {result5['prix_min']:,.0f} - {result5['prix_max']:,.0f} DT")
else:
    print(f"\n❌ Test 5 échoué: {result5['error']}")

🚗 PRÉDICTION DU PRIX

📋 INFORMATIONS DU VÉHICULE:
   • Marque:           MERCEDES
   • Modèle:           Classe E
   • Année:            2015
   • Kilométrage:      180,000 km
   • Énergie:          Diesel
   • Boîte:            Automatique
   • Puissance:        11 CV
🔧 TRANSFORMATION DES DONNÉES
📅 Année actuelle: 2025
📅 Année véhicule: 2015
📅 Âge calculé: 10 ans

✅ Features engineered créées:
   • Km_par_Age: 16,364 km/an
   • Log_Km: 12.10
   • Is_Luxury: Oui
   • Age_Category: Ancien
   • Puissance_Age_Ratio: 1.00
   • Brand_Category: Premium_European

✅ DataFrame créé: 23 colonnes
   📋 Ordre des colonnes vérifié avec training notebook
   📋 Features: Age, Kilometrage, Puissance_Fiscale, Km_par_Age, Log_Km, Is_Luxury,
                Puissance_Age_Ratio, Boite_Auto, Marque_encoded, +14 One-Hot

🔮 Prédiction en cours...
   • Prix prédit par le modèle: 108,409 DT

💰 RÉSULTAT DE LA PRÉDICTION

🎯 Prix estimé:     108,409 DT

📊 Fourchette:      97,568 DT - 119,249 DT
   (±10% d'incertitu

## 📊 5. COMPARAISON DES RÉSULTATS

In [38]:
# Créer un DataFrame avec tous les résultats
results_df = pd.DataFrame([
    {
        'Test': 'Test 1',
        'Véhicule': f"{result1['marque']} {result1['modele']}",
        'Année': result1['annee'],
        'Âge': result1['age'],
        'Km': f"{result1['kilometrage']:,.0f}",
        'Énergie': result1['energie'],
        'Boîte': result1['boite_vitesses'],
        'Prix (DT)': f"{result1['prix_predit']:,.0f}"
    },
    {
        'Test': 'Test 2',
        'Véhicule': f"{result2['marque']} {result2['modele']}",
        'Année': result2['annee'],
        'Âge': result2['age'],
        'Km': f"{result2['kilometrage']:,.0f}",
        'Énergie': result2['energie'],
        'Boîte': result2['boite_vitesses'],
        'Prix (DT)': f"{result2['prix_predit']:,.0f}"
    },
    {
        'Test': 'Test 3',
        'Véhicule': f"{result3['marque']} {result3['modele']}",
        'Année': result3['annee'],
        'Âge': result3['age'],
        'Km': f"{result3['kilometrage']:,.0f}",
        'Énergie': result3['energie'],
        'Boîte': result3['boite_vitesses'],
        'Prix (DT)': f"{result3['prix_predit']:,.0f}"
    },
    {
        'Test': 'Test 4',
        'Véhicule': f"{result4['marque']} {result4['modele']}",
        'Année': result4['annee'],
        'Âge': result4['age'],
        'Km': f"{result4['kilometrage']:,.0f}",
        'Énergie': result4['energie'],
        'Boîte': result4['boite_vitesses'],
        'Prix (DT)': f"{result4['prix_predit']:,.0f}"
    },
    {
        'Test': 'Test 5',
        'Véhicule': f"{result5['marque']} {result5['modele']}",
        'Année': result5['annee'],
        'Âge': result5['age'],
        'Km': f"{result5['kilometrage']:,.0f}",
        'Énergie': result5['energie'],
        'Boîte': result5['boite_vitesses'],
        'Prix (DT)': f"{result5['prix_predit']:,.0f}"
    }
])

print("="*100)
print("📊 TABLEAU COMPARATIF DES PRÉDICTIONS")
print("="*100)
display(results_df)
print("\n✅ Tous les tests ont réussi!")

📊 TABLEAU COMPARATIF DES PRÉDICTIONS


,Test,Véhicule,Année,Âge,Km,Énergie,Boîte,Prix (DT)
0,Test 1,BMW Série 1,2020,5,"130,000",Essence,Automatique,"133,828"
1,Test 2,PEUGEOT 208,2021,4,"119,000",Diesel,Manuelle,"49,204"
2,Test 3,Toyota Corolla,2025,0,50,Hybride,Automatique,"145,429"
3,Test 4,MG ZS,2022,3,"30,000",Essence,Automatique,"68,261"
4,Test 5,MERCEDES Classe E,2015,10,"180,000",Diesel,Automatique,"108,409"



✅ Tous les tests ont réussi!


## ✅ FIN DU PIPELINE

Le pipeline est maintenant prêt à être utilisé en production!

**Prochaines étapes suggérées:**
1. Déployer le pipeline dans une API REST (FastAPI, Flask)
2. Créer une interface web pour les utilisateurs finaux
3. Ajouter des logs pour tracker les prédictions
4. Implémenter un système de feedback pour améliorer le modèle